# Week 10 weekend work
This is based off the "N Body" Jupyter Notebook written by Yihan, and I've rewritten it OOP style. Still needs a lot of cleaning and optimization, but this is a more polished version of how it will work.

In [141]:
#################################
# rewriting Yihan's work as OOP #
#################################

import numpy as np 
import random
from matplotlib import pyplot as plt
from matplotlib import animation as anim
from mpl_toolkits.mplot3d import Axes3D

class yihanParticle():
    """
    A duplicated particle class for testing with the
    N-body code. Particles are assumed to have constant
    mass, and only have a position and velocity.
    ...
    Attributes
    ----------
    p : list
        List with three elements, representing the
        particle's position in 3D space.
        
    d : list
        List with three elements, representing the
        particle's direction vector in 3D space.

    Methods
    -------
    getPos()
        Returns position of the particle.
        
    getDir()
        Returns direction vector of the particle.

    getMagnitude()
        Returns magnitude of the direction vector.
        
    setPos(newPos)
        Updates position of the particle.
        
    setDir(newDir)
        Updates direction vector of the particle.
    """

    def __init__(self, position, direction):
        self._p = position
        self._d = direction
    
    def __str__(self):
        return f"Current position: {self._p}. Current direction: {self._d}."
    
    def getPos(self):
        return self._p
    
    def getDir(self):
        return self._d

    def getMagnitude(self):
        return np.linalg.norm(self._p)
    
    def setPos(self, newPos):
        self._p = newPos
        
    def setDir(self, newDir):
        self._d = newDir

##################################
# end of our yihanParticle class #
##################################

# yPS short for yihanParticleSimulator
class yPS():
    
    def __init__(self, n, dims, seed):
        self.n = n
        self.g = random.seed(seed)
        self.lbound = dims[0]
        self.ubound = dims[1]
        particlePositions = []
        for i in range(n):
            particlePositions.append(
                [random.randrange(self.lbound, self.ubound),
                 random.randrange(self.lbound, self.ubound),
                 random.randrange(self.lbound, self.ubound)])
        # initial direction vector of 0
        self.particles = [yihanParticle(particlePositions[i], [0, 0, 0]) for i in range(n)]
        
    def __str__(self):
        return f"System of {self.n} particles. Bounded by [{self.lbound}, {self.ubound}]."
    
    def printAllPos(self):
        for i in self.particles:
            print(i)
    
    def printThisPos(self, n):
        print(self.particles[n])
    
    # generates list of accelerations
    # O(n^2) at present - to optimize.
    def accGen(self):
        allAcc = [[] for i in range(self.n)]
        finalAcc = [0 for i in range(self.n)]
        for i in range(self.n):
            temp = []
            for j in range(self.n):
                temp.append(self.pwAcc(self.particles[i], self.particles[j]))
            allAcc[i] = temp
        for i in range(self.n):
            #finalAcc.append(allAcc[i])
            xSum = sum(j[0] for j in allAcc[i])
            ySum = sum(j[1] for j in allAcc[i])
            zSum = sum(j[2] for j in allAcc[i])
            finalAcc[i] = [xSum, ySum, zSum]
        return finalAcc
    
    # updates velocities and positions of the particles
    # hardcoded timestep of 0.01s
    def updateAll(self):
        accs = self.accGen()
        for i in range(self.n):
            tPar = self.particles[i]
            timedAcc = [accs[i][j] * 0.1 for j in range(3)]
            timedVel = [tPar.getDir()[k] * 0.1 for k in range(3)]
            tempAcc = [accs[i][l] * (0.5 * (0.1 ** 2)) for l in range(3)]
            newPos = [tPar.getPos()[a] + timedVel[a] + tempAcc[a] for a in range(3)]
            newVel = [tPar.getDir()[a] + timedAcc[a] for a in range(3)]
            self.particles[i].setPos(newPos)
            self.particles[i].setDir(newVel)
            
    # pairwise acceleration of p1 towards p2
    def pwAcc(self, p1, p2):
        if p1 == p2:
            return [0.0, 0.0, 0.0]
        else:
            radius = self.getDist(p1, p2) / 2
            vec = self.pToP(p1, p2)
            for i in range(3):
                vec[i] = vec[i] / (radius ** 3)
            return vec
        
    # direction vector from p1 to p2
    def pToP(self, p1, p2):
        if p1 == p2:
            return [0, 0, 0]
        else:
            vec = [0, 0, 0]
            for i in range(3):
                vec[i] = p1.getPos()[i] - p2.getPos()[i]
            return vec
    
    # distance between two particles
    def getDist(self, p1, p2):
        if p1 == p2:
            return 0.0
        else:
            vec = [0, 0, 0]
            for i in range(3):
                vec[i] = p1.getPos()[i] - p2.getPos()[i]
            return np.linalg.norm(vec)
        
    # a basic (and wrong) kernel
    # if distance more than 1, effect is 0
    # if distance less than 1, effect is 1-distance^2
    # not needed with pairwise acceleration present
    # not currently used
    def effect(self, p1, p2):
        if p1 == p2:
            return 0.0
        elif yPS.getDist(p1, p2) > 1:
            return 0.0
        else:
            scale = yPS.getDist(p1, p2)
            return 1 - (scale ** 2)
        
########################
# end of the yPS class #
########################

def plot_n_body(particles, accuracy, time):
# particles: a 2D list storing all the infomation of the particles 
# accuracy: delta t
# time: total length of time that pass 
    data_list=[]
    n=len(particles)
    iteration_no=int(time/accuracy)

    # 1: data generation, expected out put is data_list, a 3D array 
    for i in range(iteration_no):
        # 1.1: stroing particles information 
        data_list.append(particles[:,1:4].copy())
        # 1.2: update particles information 
        particles=update_particles(particles, accuracy)
    data_list=np.array(data_list)

    # 2: plotting 
    ax = plt.axes(projection='3d')
    for i in range(n):
        x=data_list[:,i,0]
        y=data_list[:,i,1]
        z=data_list[:,i,2]
        ax.plot3D(x, y, z)
    plt.show()

In [146]:
y1 = yihanParticle([1.5,1,1], [0,0,0])
y2 = yihanParticle([2,1,1], [0,0,0])
#print(pwAcc(y1, y1))
yp = yPS(5, [0, 10], 2)
#print(yp.pwAcc(yp.particles[1], yp.particles[2]))
print(yp)
#yp.printAllPos()

#print(yp.accGen())

yp.printThisPos(1)
for i in range(50):
    yp.updateAll()
# after 5 seconds
yp.printThisPos(1)
for i in range()

System of 5 particles. Bounded by [0, 10].
Current position: [5, 2, 4]. Current direction: [0, 0, 0].
Current position: [7.399213103617373, -0.34194869261969263, 2.772599261597456]. Current direction: [0.7582170949974327, -0.7282674018998857, -0.4706013637594539].


# Week 10 work

In [36]:
import numpy as np
import random

class randomPArray():
    """
    Particle array has the following attributes:
    1. An array of n particles
    2. Lots of functions to work on the particles
    """
    
    def __init__(self, n, dims, seed):
        """ Initializes a new random particle array.
        
        Parameters
        ----------
        n : int
            Number of particles in our particle array.
        dims : list
            A list with two elements - the range in which
            the particles can exist (think of this as the
            bounds of the simulation).
        seed : int
            Used as input for a pseudorandom number generator.
            For checking purposes.
        """
        self.g = random.seed(seed)
        self.lbound = dims[0]
        self.ubound = dims[1]
        pPositions = []
        for i in range(n):
            pPositions.append(
                [random.randrange(self.lbound, self.ubound),
                 random.randrange(self.lbound, self.ubound),
                 random.randrange(self.lbound, self.ubound)])
        self.particles = [particle(pPositions[i], [100, 0, 0]) for i in range(n)]
        
    def __str__(self):
        return f"Particles: {str([i.getPos() for i in self.particles])}"
    
    def getBounds(self):
        return [self.lbound, self.ubound]
    
    def calculateNewVel(self):
        # TO DO

        return [0, 0, 0]
    
    def updateParticles(self):
        """ Updates the positions and velocities of the particles."""
        # uncomment when I figure out how to calculate the new velocities
        # newVelocities = self.calculateNewVel()
        # TO DO
        for i in range(len(self.particles)):
            self.particles[i].updatePos()
        #    self.particles[i].newDir(newVelocities[i])
            
    #def gg(self):
    #    return self.particles[0]
        

class particle():
    """
    A class used to represent a particle. The particle
    is assumed to have constant mass.

    ...

    Attributes
    ----------
    p : list
        List with three elements, containing the x, y,
        and z coordinates of the particle.
    d : list
        List with three elements, containing the x, y,
        and z coordinates of the particle's direction
        vector.

    Methods
    -------
    getPos()
        Returns position of the particle

    getMagnitude()
        Returns magnitude of the direction vector
        
    getVel()
        Returns magnitude of the direction vector, same as
        above.
        
    getDir()
        Returns unit vector in the same direction as the
        direction vector.
        
    deltaP()
        Returns the change in position of the particle given
        it's current velocity vector.
        
    updatePos()
        Updates the position of the particle using the change
        in position from deltaP().
        
    newPos(newPos)
        Testing function to manually update the position
        of the particle.
        
    newDir(newDir)
        Testing function to manually update the direction
        vector of the particle.
    """
    
    def __init__(self, position, direction):
        """ Initializes a new particle.
        
        Parameters
        ----------
        position : list
            List with three elements, containing the x, y,
            and z coordinates of the particle.
        direction : list
            List with three elements, containing the x, y,
            and z coordinates of the particle's direction
            vector.
        """
        self.p = position
        self.d = direction
    
    def __str__(self):
        return f"Current position: {self.p}. Current direction: {self.d}."
    
    def getPos(self):
        return self.p

    def getMagnitude(self):
        return np.linalg.norm(self.d)
    
    def getVel(self):
        return self.getMagnitude()
    
    def getDir(self):
        return self.d/self.getMagnitude()
    
    def deltaP(self):
        """Uses t = 0.01 as a timestep."""
        temp = self.d.copy()
        for i in range(3):
            temp[i] = self.d[i] * 0.01
        return temp
        
    def updatePos(self):
        temp = self.deltaP()
        for i in range(3):
            self.p[i] += temp[i]
        
    def newPos(self, newPos):
        self.p = newPos
        
    def newDir(self, newDir):
        self.d = newDir

In [37]:
p1 = particle([0, 0, 0], [1, 1, 1])
print(p1)
p1.updatePos()
print(p1)

Current position: [0, 0, 0]. Current direction: [1, 1, 1].
Current position: [0.01, 0.01, 0.01]. Current direction: [1, 1, 1].


In [38]:
arr1 = randomPArray(5, [0, 10], 2)
print(arr1.getBounds())
print(arr1)

arr2 = randomPArray(3, [0, 10], 2)
print(arr2)
#print(arr2.gg())
arr2.updateParticles()
print(arr2)

[0, 10]
Particles: [[0, 1, 1], [5, 2, 4], [4, 9, 3], [9, 0, 9], [2, 6, 6]]
Particles: [[0, 1, 1], [5, 2, 4], [4, 9, 3]]
Particles: [[1.0, 1.0, 1.0], [6.0, 2.0, 4.0], [5.0, 9.0, 3.0]]
